# INTRODUCIR EL VEHICULO Y GUARDAR FOTOS DEL SENSOR
María Rodríguez Palomo

#### IMPORTAMOS LAS LIBRERIAS QUE NECESITAMOS

In [1]:
import glob
import os
import sys
import carla
import random
import time

#### CONECTAMOS CON EL SERVIDOR Y CREAMOS EL MUNDO

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
client.load_world("Town05")
world = client.get_world()

#### CREAMOS EL VEHICULO

In [9]:
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle'))
if bp.has_attribute('color'):
    color = random.choice(bp.get_attribute('color').recommended_values)
    bp.set_attribute('color', color)
transform = world.get_map().get_spawn_points()[2]
vehicle = world.spawn_actor(bp, transform)

In [10]:
actor_list = []
actor_list.append(vehicle)
print('creado %s' % vehicle.type_id)
vehicle.set_autopilot(True)

creado vehicle.mercedes.coupe


#### CREAMOS LA CÁMARA

In [11]:
camera_bp = blueprint_library.find('sensor.camera.depth')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
actor_list.append(camera)
print('creado %s' % camera.type_id)
cc = carla.ColorConverter.LogarithmicDepth
camera.listen(lambda image: image.save_to_disk('_out/%06d.png' % image.frame, cc))       

creado sensor.camera.depth


#### OBTENEMOS LA LOCALIZACIÓN DEL COCHE

In [12]:
location = vehicle.get_location()
vehicle.set_location(location)
print('vehículo movido a %s' % location)      

vehículo movido a Location(x=-54.605305, y=-23.803663, z=0.197122)


#### INCLUIMOS VEHICULOS

In [13]:
transform.location += carla.Location(x=40, y=-3.2)
transform.rotation.yaw = -180.0
for _ in range(0, 10):
    transform.location.x += 8.0
    bp = random.choice(blueprint_library.filter('vehicle'))
    npc = world.try_spawn_actor(bp, transform)
    if npc is not None:
        actor_list.append(npc)
        npc.set_autopilot(True)
        print('creado %s' % npc.type_id)

creado vehicle.lincoln.mkz_2020
creado vehicle.diamondback.century
creado vehicle.bmw.grandtourer
creado vehicle.mercedes.sprinter
creado vehicle.yamaha.yzf


#### DESTRUIMOS

In [14]:
print('destruyendo los actors')
camera.destroy()
client.apply_batch([carla.command.DestroyActor(x) for x in actor_list])
print('destruidos')

destruyendo los actors
destruidos
